#### Задание № 1

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

In [30]:
#!wget 'https://drive.google.com/uc?export=download&id=1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ' -O WA_Fn-UseC_-Telco-Customer-Churn.csv


In [31]:
import pandas as pd
import numpy as np


df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [32]:
df.gender.value_counts(normalize=True)


Male      0.504756
Female    0.495244
Name: gender, dtype: float64

#### 2. Какое количество уникальных значений у поля InternetService?

In [33]:
df.InternetService.nunique()


3

#### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [34]:
df.TotalCharges.describe()


count     7043
unique    6531
top           
freq        11
Name: TotalCharges, dtype: object

In [35]:
df.TotalCharges.unique()


array(['29.85', '1889.5', '108.15', ..., '346.45', '306.6', '6844.5'],
      dtype=object)

Видим, что признак TotalCharges имеет внутри себя строковые значения.

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

#### 4. Сделайте замену значений поля PhoneService на числовые (Yes->1, No->0)

In [36]:
y_n_dict = {
    'Yes': 1,
    'No': 0
}
df['PhoneService'] = df['PhoneService'].map(y_n_dict)
df.PhoneService


0       0
1       1
2       1
3       0
4       1
       ..
7038    1
7039    1
7040    0
7041    1
7042    1
Name: PhoneService, Length: 7043, dtype: int64

#### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [37]:
df['TotalCharges'] = df['TotalCharges'].map(
    lambda x: float(x) if x != ' ' else np.nan).astype('float32')
df['TotalCharges'].dtype


dtype('float32')

In [38]:
df['TotalCharges'] = df['TotalCharges'].map(
    lambda x: float(x) if x != ' ' else np.nan).astype('float32')
df['TotalCharges'].dtype


dtype('float32')

#### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [39]:
df['Churn'] = df['Churn'].map(y_n_dict)
df.Churn


0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

#### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport на числовые (Yes -> 1, No -> 0, No internet service->0)

In [40]:
y_n_dict.update({'No internet service': 0})


In [41]:
for col in ['StreamingMovies', 'StreamingTV', 'TechSupport']:
    df[col] = df[col].map(y_n_dict)


Заполните пропуски в поле PhoneService значением 0

In [42]:
df['PhoneService'].fillna(0, inplace=True)


#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [43]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges',
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

df = df[columns]
df.head(3)


,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.850000,0,0,0,0
1,Male,34,1,1889.500000,0,0,0,0
2,Male,2,1,108.150002,0,0,0,1


#### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [44]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges',
            'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True, random_state=42, stratify=y)


#### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [45]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline


class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]


class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(
            X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


gender = Pipeline([
    ('selector', ColumnSelector(key='gender')),
    ('ohe', OHEEncoder(key='gender'))
])


#### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [46]:
gender.fit_transform(X_train, y_train)


,gender_Female,gender_Male
5557,1,0
2270,1,0
6930,1,0
2257,1,0
898,1,0
...,...,...
4250,0,1
1488,0,1
6303,1,0
2710,1,0


#### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

Вопрос - в каких случаях это может быть полезно?

In [47]:
from sklearn.preprocessing import StandardScaler

tenure = Pipeline([
    ('selector', NumberSelector(key='tenure')),
    ('standard', StandardScaler())
])


In [48]:
tenure.fit_transform(X_train)


array([[-1.11472828],
       [-1.19588443],
       [-1.19588443],
       ...,
       [ 1.56342466],
       [-0.34374486],
       [-1.07415021]])

Стандартизация полезна в случаях использования алгоритмов кластеризации и классификации, поскольку преобразует значения прознаков к нормальному распределению со средним равным 0 и стандартным отклонением равным 1, при этом она лучше подходит для больших выборок, распределение значений признаков в которых из-за большого размера выборки стремится к нормальному.

#### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [49]:
totalcharges = Pipeline([
    ('selector', NumberSelector(key='TotalCharges')),
    ('standard', StandardScaler())
])


In [50]:
totalcharges.fit_transform(X_train)


array([[-0.837938 ],
       [-0.909176 ],
       [-0.9110082],
       ...,
       [ 2.3568413],
       [-0.7949466],
       [-0.9578821]], dtype=float32)

Объединение всех "кубиков" очень легко сделать таким образом

In [51]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
    ('selector', ColumnSelector(key=['PhoneService',
                                     'StreamingMovies', 'StreamingTV',
                                     'TechSupport']))
])


In [58]:
feats = FeatureUnion([('tenure', tenure),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])


На этом этапе что мы сделали:

написали преобразователь поля gender, который делает OHE кодирование
написали преобразователь для поля tenure, который нормирует и центрирует его
повторили п. 2 для поля TotalCharges
для всех остальных просто взяли признаки как они есть, без изменений
У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [59]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])

pipeline.fit(X_train, y_train)


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key=['PhoneService',
                                                                                      'StreamingMovies',
                                                                                      'StreamingTV',
                                                                                      'TechSupport']))])),
                                                ('gender',
                                                 Pipeline(steps=[('selector',
                                                                  ColumnSelector(key='gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='gender'))]))])),
                ('classifier', RandomForestClassifier(random_state=42))])

#### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [60]:
y_prob = pipeline.predict_proba(X_test)
y_prob


array([[0.91877165, 0.08122835],
       [0.81277425, 0.18722575],
       [0.02785714, 0.97214286],
       ...,
       [0.84748139, 0.15251861],
       [0.92628571, 0.07371429],
       [1.        , 0.        ]])

#### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [61]:
y_pred = pipeline.predict(X_test)
y_pred


array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [62]:
from sklearn.metrics import roc_auc_score, log_loss

ROC_AUC = roc_auc_score(y_test, y_pred)
LOG_LOSS = log_loss(y_test, y_prob)
print(f'ROC_AUC = {ROC_AUC}\nLOG_LOSS = {LOG_LOSS}')


ROC_AUC = 0.6260934083099032
LOG_LOSS = 1.066456596913147


Сохраним наш пайплайн

In [63]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)